In [452]:
"""
    Objective 1: Lab data
        - Plot the data
        - Check average performance (binnned) increases w/ delta -> RM-ANOVA
        - Both for orientation and color
    Objective 2: mturk data
        - Bootstrap samples to compute standard error
        - relevant data: check average performance (binned) increases w/ delta -> RM-ANOVA
        - Irrelevant data: check average performance (binned) is 
            significantly different from chance -> one-sided t-test
    Objective 3: Lab vs. mturk data (relevant)
        - Check mturk data is significanlty worse than lab data -> two-sided t-test
"""
import fnmatch # file name matching
import os
from scipy.io import loadmat

from pandas import DataFrame, Series
import pandas as pd
import scipy as sp
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib

#1. Laboratory experiment
## Change localization w/ one orientation change and one color change (N = 5)
##1.1 Load files

In [453]:
""" Define path name and subject names """
pathname = 'one_change_lab/'
file_names = os.listdir(pathname)
subject_names = ['HJK','HS0','LLT','WK0','WZ0']

In [454]:
""" Create a DataFrame for all subject data """
index = 1
# Loop over subjects (subject_names)
df = DataFrame()
for index in [1,2]:
    temp2 = DataFrame()
    for subject_name in subject_names:
        # Create a tag to find files based on a subject name
        subject_tag = subject_name + '6_6_' + str(index)
        # Filter the files
        subject_files = fnmatch.filter(file_names, subject_tag + '*.mat')
        # Loop over filtered files
        temp1 = DataFrame()
        for subject_file in subject_files:
            # Convert a MATLAB matrix to a DataFrame
            temp0 = DataFrame(loadmat(pathname+subject_file)['datamatrix'])
            # Concatenate DataFrames
            temp1 = pd.concat([temp1,temp0],ignore_index=True)
        # Create a column to assign subject names
        temp1['Name'] = DataFrame([subject_name]*len(temp1))
        # Concatenate
        temp2 = pd.concat([temp2,temp1],ignore_index=True)
        # Create a column to assign experiment index
        if index == 1:
            temp2['Feature'] = DataFrame(['Ori']*len(temp2))
        elif index == 2:
            temp2['Feature'] = DataFrame(['Col']*len(temp2))
    # Concatenate all subject data to create a single DataFrame            
    df = pd.concat([df,temp2],ignore_index=True)

In [455]:
df.loc()

###<font color='red'>Orientation: delta = column 1, Color: delta = column 2

In [456]:
df_temp = df[['Name','Feature',1,2,8]]
df_ori = df_temp.loc[:len(df_temp)/2-1,['Name','Feature',1,8]]
df_col = df_temp.loc[len(df_temp)/2:,['Name','Feature',2,8]]
print df_ori.tail()
print df_col.head()

     Name Feature          1  8
2995  WZ0     Ori -62.423398  1
2996  WZ0     Ori  65.431755  1
2997  WZ0     Ori   2.256267  0
2998  WZ0     Ori -48.885794  1
2999  WZ0     Ori -80.473538  0
     Name Feature    2  8
3000  HJK     Col   78  1
3001  HJK     Col  314  0
3002  HJK     Col  199  0
3003  HJK     Col  101  1
3004  HJK     Col   66  1


In [457]:
# Assign column names
df_ori.columns = ['Subject', 'Feature','Delta','Correct']
df_col.columns = ['Subject', 'Feature','Delta','Correct']

##1.2 Remap the delta for color data

In [458]:
"""
    Remapping color change magnitude
    Original range: range(1,361)
    Remap:
        1, ... 180: delta
        181, ..., 360: 360 - delta
    df['c'+str(i+1)] = df['C'+str(i+1)].apply((lambda x: hextorgb(x)))

"""
def remap_colors(old_delta):
    if old_delta <= 180:
        new_delta = old_delta
    else:
        new_delta = 360 - old_delta
    if new_delta == 0:
        new_delta = 0.0001
    return new_delta/2
# remap
df_col.Delta = df_col.loc[:,'Delta'].apply((lambda x: remap_colors(x)))

In [462]:
df_col.describe()

,Delta,Correct
count,3000.000000,3000.000000
mean,45.836333,0.716667
std,26.191319,0.450692
min,0.000050,0.000000
25%,23.000000,0.000000
50%,46.000000,1.000000
75%,68.500000,1.000000
max,90.000000,1.000000


In [463]:
# Concatenate
df_short = pd.concat([df_ori,df_col],ignore_index=True)
df_short.head()

,Subject,Feature,Delta,Correct
0,HJK,Ori,88.997214,1
1,HJK,Ori,-15.793872,0
2,HJK,Ori,85.487465,1
3,HJK,Ori,-56.406685,1
4,HJK,Ori,-57.409471,1


##1.3 Bin the data based on the change magnitude (delta)

###<font color='red'>Adding new columns to a df -> Use df.loc[:,new_column_name] = value

###<font color='red'>Selecting cells based on location  -> Use df.iloc[list or int,list or int] = value

In [464]:
"""
    Objective: Binning the data
    1. Get absolute values of delta
    2. Create index for delta
    3. Create an column
    4. Groupby
"""
#1. Get absolute values of delta and add it as a new column
df_short.loc[:,'AbsDelta']=abs(df_short['Delta'].values)

In [465]:
#2. Creat a new index based on bin size
def label_bin(data,n_bins):
    # takes data and bin size as input, and gives labels to each data point based on histogram results
    bins = np.linspace(round(min(data)),round(max(data)),n_bins)
    # right=True: the intervals include the right or the left bin edge. 
    # Digitize default behavior (right==False): does not include right edge
    inds = np.digitize(data, bins, right=True)
    return bins, inds

bins, inds = label_bin(df_short.AbsDelta,10)
print 'bins=' + str(bins)

# Double check if the sum of the inds in each bin is the number of the total data points
tot = 0
for i in range(min(inds),max(inds)+1):
    print 'ind(' + str(i) + ')=' + str(np.shape(np.where(inds==i))[1])
    tot += np.shape(np.where(inds==i))[1]
print 'total = ' + str(tot)

bins=[  0.  10.  20.  30.  40.  50.  60.  70.  80.  90.]
ind(1)=633
ind(2)=690
ind(3)=635
ind(4)=679
ind(5)=681
ind(6)=636
ind(7)=695
ind(8)=637
ind(9)=714
total = 6000


In [466]:
#3. Add this as a new column
df_short.loc[:,'DeltaInd']=inds

In [467]:
df_short.head()

,Subject,Feature,Delta,Correct,AbsDelta,DeltaInd
0,HJK,Ori,88.997214,1,88.997214,9
1,HJK,Ori,-15.793872,0,15.793872,2
2,HJK,Ori,85.487465,1,85.487465,9
3,HJK,Ori,-56.406685,1,56.406685,6
4,HJK,Ori,-57.409471,1,57.409471,6


##1.4 Compute summary statistics

In [491]:
"""
    Use Pandas groupby to compute statistics easily
"""
grouped = df_short.groupby(['Feature','DeltaInd'])
df_mean = grouped.mean()
df_ste = grouped.std()/np.sqrt(5)